# Cálculo do TF-IDF para os termos dos textos descritivos

## Tokenização dos textos

In [1]:
# Imports
import nltk
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize 

import pandas as pd

#import spacy
import string

In [2]:
## Extraindo frequencias de termos dos textos descritivos do dataset.

In [3]:
df = pd.read_json('data_files/merged_datasets.json', orient='split')

In [4]:
corpus = df['Text']

In [5]:
# Apagando a variável df porque não será mais utilizada.
del df

In [6]:
# Quebrando cada texto em uma lista de sentenças.
corpus_sent_tokenized = [sent_tokenize(texto) for texto in corpus]

In [7]:
# Apagando a variável corpus porque não será mais utilizada.
del corpus

In [8]:
# Quebrando as sentenças em palavras, de modo que cada sentença seja transformada em lista de palavras.
corpus_word_tokenized = [[word_tokenize(sent) for sent in text] for text in corpus_sent_tokenized]

In [9]:
# Apagando a variável corpus_sent_tokenized porque não será mais utilizada
del corpus_sent_tokenized

## Remoção das stopwords e pontuações

In [10]:
english_stops = set(stopwords.words('english'))

In [11]:
def remove_stopwords_e_pontuacao(lista_words):
    tokens_limpos = [word.lower() for word in lista_words if (word.lower() not in english_stops and word not in string.punctuation)]
    return tokens_limpos

In [12]:
corpus_limpo = [[remove_stopwords_e_pontuacao(sent) for sent in text] for text in corpus_word_tokenized]

In [13]:
# Apagando a variável corpus_word_tokenized porque não será mais utilizada.
del corpus_word_tokenized

In [14]:
# Faz-se necessário juntar todas as palavras de cada texto em uma lista única, para que seja feita a contagem
def abre_listas(lista_de_listas):
    lista_dummy = []
    for lista in lista_de_listas:
        for elemento in lista:
            lista_dummy.append(elemento)
    return lista_dummy

In [15]:
doc = abre_listas(corpus_limpo[0])

In [16]:
documentos = [abre_listas(doc) for doc in corpus_limpo]

In [17]:
# Apagando a variável corpus_word_tokenized porque não será mais utilizada.
del corpus_limpo

## Stemming e Lemmatization

### Stemming

In [18]:
from nltk.stem import PorterStemmer
from nltk.stem import RegexpStemmer

In [19]:
porter_stemmer = PorterStemmer()

In [20]:
porter_stemmer.stem('cooking')

'cook'

In [21]:
porter_stemmer.stem('cookery')

'cookeri'

In [22]:
regexp_stemmer = RegexpStemmer('ing')

In [23]:
porter_stemmer.stem(regexp_stemmer.stem('cooking'))

'cook'

In [24]:
porter_stemmer.stem(regexp_stemmer.stem('activation'))

'activ'

In [25]:
porter_stemmer.stem(regexp_stemmer.stem('activating'))

'activat'

In [26]:
# Aplicando stemming, isto é, retirada de sufixos, para cada palavra na lista de documentos
docs_stemmed = [[porter_stemmer.stem(regexp_stemmer.stem(word)) for word in doc] for doc in documentos]

In [27]:
len(docs_stemmed)

3321

### Lemmatization

In [28]:
nltk.download('wordnet')

from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package wordnet to /home/rafael/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [29]:
wordnet_lemmatizer = WordNetLemmatizer()

In [30]:
print(wordnet_lemmatizer.lemmatize('mice'))
print(wordnet_lemmatizer.lemmatize('cacti'))  # plural da palavra cactus - cactuses (inglês) ou cacti (latin)
print(wordnet_lemmatizer.lemmatize('horses'))
print(wordnet_lemmatizer.lemmatize('wolves'))

mouse
cactus
horse
wolf


In [31]:
# Aplicando lemmatization, isto é, a aglutinação das palavras com o mesmo radical comum. (útlima?)
docs_stemmed_lemmatized = [[wordnet_lemmatizer.lemmatize(word) for word in doc] for doc in documentos]

In [ ]:
# Apagando variáveis que não será mais utilizada
del documentos
del docs_stemmed

## Cálculo do TF-IDF

### Cálculo do TF

In [32]:
# Função para calcular Term-frequency (frequencia do termo)
def TermFreq(documento, palavra):
    doc_length = len(documento)
    ocorrencias = len([w for w in documento if w == palavra])
    return ocorrencias / doc_length

In [33]:
TermFreq(docs_stemmed_lemmatized[0], 'cyclin')

0.02197530864197531

In [34]:
TermFreq(docs_stemmed_lemmatized[0], 'cancer')

0.002962962962962963

In [35]:
palavras_corpus = abre_listas(docs_stemmed_lemmatized)

In [36]:
# Criação de uma lista única contendo todas as palavras dos documentos para contagem dos termos no corpus
set_palavras_corpus = set(palavras_corpus)

In [37]:
len(set_palavras_corpus)

276083

In [38]:
len(palavras_corpus)

21194984

### Estabelecendo um contador para apurar a frequência das palavras

In [39]:
from collections import Counter

In [40]:
%%time
contador = Counter(palavras_corpus)

CPU times: user 2.64 s, sys: 8.01 ms, total: 2.65 s
Wall time: 2.65 s


In [41]:
len(contador)

276083

In [42]:
contador['revealed']

12185

In [43]:
chaves = contador.keys()

In [44]:
'kinase' in chaves

True

In [45]:
# Versão 1 implementada totalmente em Python

# Criamos um corpus Bag of words
def cria_dict(set_palavras, lista_docs):
    output = {}
    for word in set_palavras:
        output[word] = 0
        for doc in lista_docs:
            if word in doc:
                output[word] += 1
    return output

In [46]:
lista_teste = docs_stemmed_lemmatized[0:10]

In [47]:
set_teste = set(abre_listas(lista_teste))

In [48]:
%%time
dict_1 = cria_dict(set_teste, lista_teste)

CPU times: user 3.38 s, sys: 3.99 ms, total: 3.38 s
Wall time: 3.38 s


In [49]:
lista_sets = [Counter(doc).keys() for doc in docs_stemmed_lemmatized]

In [50]:
lista_teste_2 = lista_sets[0:10]

In [51]:
# Versão 2 otimizada utilizando contadores.

# Criamos um corpus Bag of words
def cria_dict_2(set_palavras, lista_sets):
    output = {}
    for word in set_palavras:
        output[word] = 0
        for set in lista_sets:
            if word in set:
                output[word] += 1
    return output

In [52]:
%%time
dict_2 = cria_dict_2(set_teste, lista_teste_2)

CPU times: user 13.1 ms, sys: 0 ns, total: 13.1 ms
Wall time: 12.9 ms


A função cria_dict_2 apresentou desempenho 340x melhor do que a função cria_dict

In [53]:
%%time
# Criação do dicionário completo com todos os documentos do dataset
dict_completo = cria_dict_2(set_palavras_corpus, lista_sets)

CPU times: user 2min 8s, sys: 15.7 ms, total: 2min 8s
Wall time: 2min 8s


In [54]:
len(dict_completo)

276083

### Cálculo do IDF

In [55]:
# Função para calcular a Frequência Inversa de Documentos
def InverseDocumentFrequency(word):
    N = len(docs_stemmed_lemmatized)
    try:
        df = dict_completo[word] + 1
    except:
        df = 1
    return np.log(N/df)

In [56]:
InverseDocumentFrequency('human')

0.09467803999007461

In [57]:
InverseDocumentFrequency('mother')

2.543500814054053

In [58]:
InverseDocumentFrequency('data')

0.031195190077938376

### Cálculo do TF-IDF

In [59]:
# Função TF-IDF
def TFIDF(doc, word):
    tf = TermFreq(doc, word)
    idf = InverseDocumentFrequency(word)
    return tf * idf

In [60]:
TFIDF(docs_stemmed_lemmatized[0], 'cellular')

0.00029136573894505155

In [61]:
TFIDF(docs_stemmed_lemmatized[0], 'activating')

0.0002700738922698808

In [62]:
for word in set(docs_stemmed_lemmatized[0]):
    print(f'{word} - TF-IDF: {TFIDF(docs_stemmed_lemmatized[0], word)}')

s10b - TF-IDF: 0.001521508906746033
naf - TF-IDF: 0.001683859618010475
s10a - TF-IDF: 0.002414392788399282
e - TF-IDF: 0.00045657462797897303
regulating - TF-IDF: 0.00032019181303097615
lysates - TF-IDF: 0.0010344878034012194
peg202 - TF-IDF: 0.0018308330964979759
xq28-targeted - TF-IDF: 0.0018308330964979759
fbs - TF-IDF: 0.0007029182684981824
gap - TF-IDF: 0.0004421867986739414
a5060 - TF-IDF: 0.0018308330964979759
glutathione-sepharose - TF-IDF: 0.0016156737355186512
activity - TF-IDF: 0.0001852858405044164
spun - TF-IDF: 0.004870725415958088
coexpressing - TF-IDF: 0.002995404845573254
sirna-mediated - TF-IDF: 0.0017448445643819081
predict - TF-IDF: 0.00021205090475613764
nasal - TF-IDF: 0.0009789843197494355
x-chromosome - TF-IDF: 0.0011462432885375358
west - TF-IDF: 0.000643645187859758
γ-32p - TF-IDF: 0.0007889793421335049
combined - TF-IDF: 0.0001725641425818811
s11 - TF-IDF: 0.0016275737609897922
child - TF-IDF: 0.00046310449501882074
high-resolution - TF-IDF: 0.000643645187859

process - TF-IDF: 0.0002841469642409672
glutathione - TF-IDF: 0.000574636470514671
defined - TF-IDF: 0.00014647443478717925
total - TF-IDF: 0.00019282172907265764
ets2wt - TF-IDF: 0.0036616661929959517
detecting - TF-IDF: 0.0005213750945080804
act - TF-IDF: 0.0003550988265473133
quantification - TF-IDF: 0.0022311548477944474
inhibits - TF-IDF: 0.00025591305115190484
proposed - TF-IDF: 0.00018482804521509274
real-time - TF-IDF: 0.0004084928944405074
production - TF-IDF: 0.0003423692643422477
atypical - TF-IDF: 0.0005314034202251644
-driven - TF-IDF: 0.001217275899019457
regulate - TF-IDF: 0.0004797718581661724
differing - TF-IDF: 0.0005458931737629645
dog - TF-IDF: 0.000963784839996489
infection - TF-IDF: 0.00036153863906890723
flanking - TF-IDF: 0.0003690461735795524
conferring - TF-IDF: 0.0005275949090336504
2–5 - TF-IDF: 0.0015497323070620967
sirna - TF-IDF: 0.001402528239225596
females12 - TF-IDF: 0.0018308330964979759
novagen - TF-IDF: 0.000821950056361782
without - TF-IDF: 0.00022

## Verificando quantas palavras aparecem menos vezes no corpus todo
Intenção é eliminar valores espúrios e reduzir o tamanho do dataset gerado, uma vez que não servirão para nenhuma generalização do modelo

In [63]:
len(contador)

276083

In [72]:
min(contador.values())

1

In [73]:
max(contador.values())

339239

In [64]:
from tqdm import tqdm

In [74]:
# Verificando a qtd de palavras que podem ser eliminadas que aparecem até 100x no corpus
eliminar = [[word for word in contador.keys() if contador[word] <= n+1] for n in range(100)]

In [75]:
for n in range(len(eliminar)):
    print(f'Número de palavras que aparecem até {n+1} vez-es: {len(eliminar[n])}')

Número de palavras que aparecem até 1 vez-es: 105919
Número de palavras que aparecem até 2 vez-es: 149547
Número de palavras que aparecem até 3 vez-es: 168673
Número de palavras que aparecem até 4 vez-es: 184821
Número de palavras que aparecem até 5 vez-es: 193744
Número de palavras que aparecem até 6 vez-es: 201524
Número de palavras que aparecem até 7 vez-es: 206168
Número de palavras que aparecem até 8 vez-es: 211748
Número de palavras que aparecem até 9 vez-es: 215623
Número de palavras que aparecem até 10 vez-es: 218927
Número de palavras que aparecem até 11 vez-es: 221432
Número de palavras que aparecem até 12 vez-es: 225280
Número de palavras que aparecem até 13 vez-es: 227925
Número de palavras que aparecem até 14 vez-es: 230215
Número de palavras que aparecem até 15 vez-es: 231778
Número de palavras que aparecem até 16 vez-es: 233538
Número de palavras que aparecem até 17 vez-es: 235002
Número de palavras que aparecem até 18 vez-es: 236413
Número de palavras que aparecem até 1

In [67]:
# Eliminando palavras que aparecem 10 vezes ou menos em todo o corpus
docs_compressed = [list(set(doc) - set(eliminar[9])) for doc in tqdm(docs_stemmed_lemmatized)]

100%|██████████| 3321/3321 [01:58<00:00, 28.12it/s]


In [78]:
type(docs_compressed)

list

In [76]:
docs_compressed[1]

['downregulation',
 'cbl',
 'internalization',
 'depicted',
 'fam',
 'find',
 'ubiquitinated',
 '39',
 'pax8',
 'ca',
 'regulating',
 'lysates',
 'content',
 '27',
 'activation',
 'duplex',
 'fbs',
 'gap',
 '7.4',
 'adenocarcinoma',
 'sh3',
 '0.25',
 'substrate',
 'floor',
 'trypsin/edta',
 'briefly',
 'extracted',
 'c-abl',
 'ii',
 'activity',
 'usa',
 'vegfr',
 'construct',
 'oh',
 'activating',
 'mgcl2',
 'previously',
 'melanoma',
 'q249e',
 'room',
 '1.8',
 'louis',
 'predict',
 'divided',
 'chosen',
 'association',
 'associate',
 'proline-rich',
 'x100',
 'leading',
 'dntps',
 'among',
 'rtk',
 'site-directed',
 'combined',
 'paxillin',
 'additional',
 'background',
 'leukemia',
 '119',
 'recycling',
 'carcinogenesis',
 'copy',
 'geneamp',
 'whose',
 'efficacious',
 'vector',
 'µm',
 'santa',
 '1×',
 'phosphate',
 'g2/m',
 'juvenile',
 'dilution',
 'kras',
 'given',
 'effect',
 'overnight',
 'roche',
 'trypan',
 'possibility',
 'changed',
 'ir',
 'src',
 'dual',
 'role',
 'degrad

In [79]:
type(docs_stemmed_lemmatized)

list

In [77]:
docs_stemmed_lemmatized[1]

['abstract',
 'background',
 'non-small',
 'cell',
 'lung',
 'cancer',
 'nsclc',
 'heterogeneous',
 'group',
 'disorder',
 'number',
 'genetic',
 'proteomic',
 'alteration',
 'c-cbl',
 'e3',
 'ubiquitin',
 'ligase',
 'adaptor',
 'molecule',
 'important',
 'normal',
 'homeostasis',
 'cancer',
 'determined',
 'genetic',
 'variation',
 'c-cbl',
 'relationship',
 'receptor',
 'tyrosine',
 'kinase',
 'egfr',
 'met',
 'functionality',
 'nsclc',
 'method',
 'finding',
 'using',
 'archival',
 'formalin-fixed',
 'paraffin',
 'embedded',
 'ffpe',
 'extracted',
 'genomic',
 'dna',
 'show',
 'c-cbl',
 'mutation',
 'occur',
 'somatic',
 'fashion',
 'lung',
 'cancer',
 'c-cbl',
 'mutation',
 'mutually',
 'exclusive',
 'met',
 'egfr',
 'mutation',
 'however',
 'independent',
 'p53',
 'kras',
 'mutation',
 'normal/tumor',
 'pairwise',
 'analysis',
 'significant',
 'loss',
 'heterozygosity',
 'loh',
 'c-cbl',
 'locus',
 '22',
 'n',
 '8/37',
 'none',
 'sample',
 'revealed',
 'mutation',
 'remaining',
 '

In [68]:
len(docs_compressed)

3321

In [69]:
## Construindo um dicionário para armazenar os resultados do tf_idf por palavra, por documento

In [80]:
%%time
# Montagem do dicionário do arquivo reduzido, excluindo as palavras que aparecem menos de 10 vezes no corpus.
dict_tf_idf = {c: {word: TFIDF(doc, word) for word in doc} for (c, doc) in tqdm(enumerate(docs_compressed))}

3321it [05:13, 10.61it/s]

CPU times: user 5min 12s, sys: 588 ms, total: 5min 13s
Wall time: 5min 13s


In [81]:
import json

with open('data_files/data_tf_idf_red_v1.json', 'w') as fp:
    json.dump(dict_tf_idf, fp)
    
fp.close()

In [86]:
%%time
# Montagem do dicionário do arquivo reduzido, excluindo as palavras que aparecem menos de 10 vezes no corpus.
dict_tf_idf = {c: {word: TFIDF(doc, word) for word in doc} for (c, doc) in tqdm(enumerate(docs_stemmed_lemmatized))}

3321it [1:37:04,  1.75s/it]

CPU times: user 1h 37min 3s, sys: 1.44 s, total: 1h 37min 4s
Wall time: 1h 37min 4s


In [87]:
# Salvando o arquivo completo
#import json

with open('data_files/data_tf_idf_completo.json', 'w') as fp:
    json.dump(dict_tf_idf, fp)
    
fp.close()